In [9]:
#########################  单纯形法  ###########################

#将数据化到单纯形表中
def build(c,b,matrix):
    b = b[:,np.newaxis]      #变二维，成一列
    X = np.append(b,matrix,axis=1)   #横向合并
    c = c.astype(np.float64)
    c = np.insert(c,0,[np.nan])[np.newaxis,:]    #变二维，成一行
    X = np.append(X,c,axis=0)    #纵向合并
    return X


#寻找旋转中心并进行旋转运算
def rotation(X,base):       #X存放单纯形表，base存放基变量序号
    X = X.astype(np.float64)
    j = np.argmax(X[-1,1:])+1     #变量j进基(列号也为j)
    
    temp = [x for x in range(X.shape[0]-1) if X[x][j]>0]  #系数>0的基变量的行标
    i = temp[0]                   #i行基变量出基
    for t in temp:
        if X[t,0]/X[t,j]<X[i,0]/X[i][j]: i=t 
    
    base[i] = j    #更新base
    
    X[i] = X[i]/X[i][j]
    for k in range(X.shape[0]):
        if k==i: continue
        else: X[k] = X[k]-X[i]*X[k][j]
    return X,base


#第一阶段（获得初始可行解）
def stage1(X):
    
    lower = X.shape[1]   #列数(原变量个数+1)
    #制表
    arti = X.shape[0]-1     #行数-1
    plus1 = np.eye(arti)
    plus2 = -np.ones(arti)[np.newaxis,:]    #变二维，成一行
    plus = np.append(plus1,plus2,axis=0)    #纵向合并
    X[-1,1:] = 0
    X = np.append(X,plus,axis=1)    #横向合并
    
    #运算
    upper = X.shape[1]    #列数(原变量个数+人工变量数+1)
    base = list(range(lower,upper))   #保存基变量序号
    for i in range(X.shape[0]-1):   
        X[-1] = X[-1]+X[i]        #检验数
        
    while np.any(X[-1,1:]>0):
        X,base = rotation(X,base)
        
    if set(range(lower,upper)) & set(base) == set(): #人工变量是否全部出基
        flag = 1   #全部出基
    else: 
        flag = 0   #有变量尚未出基
        
    return X,base,flag


#第二阶段（获得得到最优解时的单纯形表）
def stage2(X,c,base):
    for i in range(X.shape[0]-1): X = np.delete(X,-1,axis=1)    #删掉人工变量对应的列
    X[-1,1:] = c[np.newaxis,:]     #变二维，成一行
    
    for i in base:   #i遍历基变量名
        row = base.index(i)
        X[-1] = X[-1]-X[row]*X[-1,i]/X[row,i]       #获得检验数
        
    while np.any(X[-1,1:]>0):
        X,base = rotation(X,base)
    return X,base


#获得最优解和最优目标函数值
def linpro(c,b,matrix):
    #获得最终单纯形表
    X = build(c,b,matrix)
    X,base,flag = stage1(X)
    if flag ==1:
        X,base = stage2(X,c,base)
    else:
        print("该问题无可行解")
        return 0,0
    
    #整理结果
    result = [0 for n in range(X.shape[1]-1)]
    for i in base:    #i遍历基变量名
        result[i-1] = X[base.index(i)][0]
    Y = np.sum(result*c)
    
    return result,Y



# c = np.array([40,50,0,0,0])
# b = np.array([30,60,24])
# mat = np.array([[1,2,1,0,0],[3,2,0,1,0],[0,2,0,0,1]])
# c = np.array([1,2,1,0])
# b = np.array([120,60])
# mat = np.array([[1,4,-2,-1],[1,1,1,0]])

# r,y = linpro(c,b,mat)
# r

In [8]:
import numpy as np
import pandas as pd
import copy
import random

#########################  最短路算法  ###########################

inf = 1000000

#定义路网类(包含路网重要信息)
class Net(object):

     def __init__(self,disnet,volumn_matrix):   
            self.disnet = disnet       #距离矩阵
            self.section = []          #路段
            self.volumn = []           #路段容量
            for i in range(disnet.shape[0]):
                for j in range(disnet.shape[1]):
                    if 0 < disnet[i][j] < inf:
                        self.section.append([i+1,j+1])
            for i in range(len(self.section)):
                point1 = self.section[i][0]      #路段起点
                point2 = self.section[i][1]      #路段终点
                self.volumn.append(volumn_matrix[point1-1][point2-1])

#求最短路
     def dijkstra(self,disnet,start,end):
            
            #初始化
            start = start-1
            end = end-1
            P = [start]
            T = [x for x in range(len(disnet)) if x!=start]
            mindis = copy.deepcopy(disnet[start])            
            
            #找出P中标号最小的点
            while len(T):
                new = T[0]
                for i in T:
                    if mindis[i] < mindis[new]: new = i   
                
                T.remove(new)
                P.append(new)
            
            #更新标号
                for i in T:
                    if mindis[new]+disnet[new][i] < mindis[i] : 
                        mindis[i] = mindis[new]+disnet[new][i]   

            #反向追踪
            route = [end]
            temp = end
            Range = [x for x in range(len(disnet))]
            while mindis[temp] > 0: 
                Range.remove(temp)
                for i in Range:
                    if mindis[i]+disnet[i][temp] == mindis[temp]:
                        route.append(i)
                        temp = i
            route.reverse()
            route = [x+1 for x in route]
           
            return route

#计算路径长度
     def cal(self,p):
        distance = 0
        p = [x-1 for x in p]
        for i in range(len(p)-1):
            distance = distance+self.disnet[p[i]][p[i+1]]
        return distance
    
#判断路段是否在路径上,在返回1，不在返回0
     def judge(self,path,section):
        for i in range(len(path)-1):
            if section == [path[i],path[i+1]]:
                return 1
        return 0

#判断路径上是否设置充电站，若有，返回遇到的第一个充电站到O点的距离，若没有，返回0
     def search_station(self,path,y,A):     #y为向量，每个元素都为01变量，对应某个
                                             #节点是否设置充电站;A为y中各元素对应节点号。
                                              #本例中，A为(2,5,6,8,10,12)
        for i in path:          #i节点号
            if i in A:
                if y[A.index(i)] == 1: 
                    return self.cal(path[:path.index(i)+1])      #返回第一个充电站到O点的距离
        return 0
        

#寻找备选路径
     def searchroute(self,start,end):
        pathset = []
        costs = []
        
        #初始化
        newdisnet = copy.deepcopy(self.disnet)
        p1 = self.dijkstra(newdisnet,start,end)
        c1 = self.cal(p1)
        pathset.append(p1)
        costs.append(c1)
        
        while len(pathset)<4:
            p = self.dijkstra(newdisnet,start,end)
            
            if p in pathset:
                #更新路阻
                p = [x-1 for x in p]
                for i in range(len(p)-1):
                    newdisnet[p[i]][p[i+1]] = newdisnet[p[i]][p[i+1]]*1.05
                continue
            else: 
                c = self.cal(p)
                pathset.append(p)
                costs.append(c)
        
#         print(pathset,costs)
        return pathset,costs

#读入路阻文件和路段容量文件
network = pd.read_csv('C:/Users/92914/iSpace/Designs/datas/traffic_analysis_1/network.csv')     
volumn = pd.read_csv('C:/Users/92914/iSpace/Designs/datas/traffic_analysis_1/volumn.csv')     
network = network.values
volumn = volumn.values
Net = Net(network,volumn)

#pathgroup第一维度：OD对，第二维度：路径，第三维度：节点
pathgroup = [Net.searchroute(1,7)[0],Net.searchroute(1,11)[0],Net.searchroute(7,1)[0],   
              Net.searchroute(7,11)[0],Net.searchroute(11,1)[0],Net.searchroute(11,7)[0]]

#pathcost第一维度：OD对，第二维度：路径
pathcost = [Net.searchroute(1,7)[1],Net.searchroute(1,11)[1],Net.searchroute(7,1)[1],    
              Net.searchroute(7,11)[1],Net.searchroute(11,1)[1],Net.searchroute(11,7)[1]]

pathgroup

[[[1, 2, 3, 7], [1, 4, 5, 7], [1, 4, 6, 7], [1, 2, 5, 7]],
 [[1, 4, 6, 9, 11],
  [1, 4, 6, 10, 12, 11],
  [1, 2, 5, 6, 10, 12, 11],
  [1, 2, 3, 7, 8, 12, 11]],
 [[7, 3, 2, 1], [7, 5, 4, 1], [7, 6, 4, 1], [7, 5, 2, 1]],
 [[7, 8, 12, 11], [7, 6, 9, 11], [7, 6, 10, 11], [7, 8, 6, 10, 11]],
 [[11, 9, 6, 4, 1],
  [11, 12, 10, 6, 4, 1],
  [11, 12, 10, 6, 5, 2, 1],
  [11, 12, 8, 7, 3, 2, 1]],
 [[11, 12, 8, 7], [11, 9, 6, 7], [11, 10, 6, 7], [11, 10, 6, 8, 7]]]

In [14]:
#########################  用单纯形法解下层模型  ###########################

ODnum = 6         #OD对数目
t0 = 10           #充电时间 
Lmax = [120,110,90,105,85,70]           #最大剩余里程
Q = [5000,8000,6000,7500,9500,6000]     #OD需求
A = [2,5,6,8,10,12]                     #潜在的设置充电桩的位置

#对于不同的充电桩设置方式，线性规划问题的价值系数、常数项、路段容量约束的系数、OD出行需
  #求约束的系数是相同的，可以放在分配函数外面进行，从而避免多次循环,加速遗传进化的过程。

#确定价值系数
c = np.array([])
for i in range(ODnum): 
    for j in range(4):
        if pathcost[i][j] > Lmax[i]:
            value = pathcost[i][j] + t0
        else: value = pathcost[i][j]
        c = np.append(c,value)

#确定常数项和系数矩阵的一部分
b = np.array([])
matrix = np.zeros((len(Net.section)+ODnum+4*ODnum,4*ODnum))

#路段容量约束
for i in range(len(Net.section)):
    b = np.append(b,Net.volumn[i])

for i in range(len(Net.section)):    #路段i
    for j in range(ODnum):           #OD对j
        for k in range(4):           #路径k
            if Net.judge(pathgroup[j][k],Net.section[i]): matrix[i][4*j+k] = 1   
                
#OD出行需求约束
rownum = len(Net.section)
for i in range(ODnum):
    b = np.append(b,Q[i])

for i in range(ODnum):          #OD对i
    for k in range(4):          #路径k
        matrix[rownum+i][4*i+k] = 1

#最大行驶里程约束对应常数项
for i in range(4*ODnum):             #所有路径i
    b = np.append(b,0)
    
y = [0,0,1,0,1,1]                  #example for y


def distribution(y,Net,ODnum,A,Lmax,pathgroup,pathcost,matrix,c,b):
    
    mat =matrix.copy()              #深拷贝matrix，使其经历此函数后内容不变

#最大行驶里程约束对应系数
    rownum = len(Net.section) + ODnum
    for i in range(4*ODnum):             #所有路径i

        #确定l_k^st
        if Net.search_station(pathgroup[i//4][i%4],y,A) != 0:
            li = Net.search_station(pathgroup[i//4][i%4],y,A)
        else:
            if pathcost[i//4][i%4] <= Lmax[i//4]: li = 0
            else: li = 800
        mat[rownum+i][i] = li-Lmax[i//4]

#标准化
    arti_number = len(Net.section) + ODnum*4
    arti_value = np.zeros((arti_number))                           #松弛变量价值系数
    c = np.append(c,arti_value)
    plus = [np.eye(len(Net.section)),np.zeros((len(Net.section)+ODnum,4*ODnum)),
            np.zeros((5*ODnum,len(Net.section))),np.eye(4*ODnum)] #松弛变量系数
    plus1,plus2 = np.append(plus[0],plus[2],axis=0),np.append(plus[1],plus[3],axis=0)
    mat = np.concatenate((mat,plus1,plus2),axis=1)
    
#求解
    if linpro(c,b,mat) !=(0,0):            #有可行解
        result,totaltime = linpro(c,b,mat)
        result = result[:4*ODnum]
        return result,totaltime
    else: return [0 for x in range(len(c))],1000000000        #无可行解,进化过程中自然淘汰


result,totaltime = distribution(y=y,Net=Net,ODnum=ODnum,A=A,Lmax=Lmax,pathgroup=pathgroup,pathcost=pathcost,matrix=matrix,c=c,b=b)


print(result,totaltime)

# matrix
# df = pd.DataFrame(mm)
# df
# df.to_csv('C:/Users/92914/iSpace/Designs/datas/traffic_analysis_1/simplex method.csv')

[1000.0, 0, 4000.0, 0.0, 4000.0, 0, 4000.0, 0.0, 500.0, 0.0, 5500.0, 0.0, 2500.0, 1000.0, 500.0, 3500.0, 2500.0, 0, 4000.0, 3000.0, 2000.0, 0.0, 3000.0, 1000.0] 6627500.0


In [15]:
#########################  用遗传算法解双层规划  ###########################

alpha = 0.5
tao = 800000

#data structure: population = [[[chromosome],adaption,match,result],...]
#chromosome为充电站设置方案，adaption为适应度，match为随机匹配的01之间的小数，
    #result为交通分配的结果即路径流量

def solve(distribution,alpha,tao,chromo,Net,ODnum,A,Lmax,pathgroup,pathcost,matrix,c,b):    #适应度函数
    result,totaltime = distribution(y=chromo,Net=Net,ODnum=ODnum,A=A,Lmax=Lmax,pathgroup=pathgroup,
                                    pathcost=pathcost,matrix=matrix,c=c,b=b)
    return result,alpha * totaltime + tao * sum(chromo) 


#冒泡排序
def sort(lists,std):      #std为排序的依据
    line = copy.deepcopy(lists)
    for i in range(len(line)-1):
        for j in range(len(line)-1-i):
            if line[j][std]>line[j+1][std]: line[j],line[j+1] = line[j+1], line[j]
    return line

#用线性同余法产生01之间均匀分布的随机数序列
def produce_series(libsize,S,a,b,c):    ##序列大小，种子，线性同余法参数
    
    library = []   #随机小数序列
    intlib = []    #随机整数序列(0/1)
    intlib5 = []   #随机整数序列(1-5)

    for i in range(libsize):
        newS = (a*S+b)%c
        R = newS/c
        intR = round(R)
        if R<0.2:intR5 = 1
        elif R<0.4:intR5 = 2 
        elif R<0.6:intR5 = 3 
        elif R<0.8:intR5 = 4
        else:intR5 = 5
        library.append(R)
        intlib.append(intR)
        intlib5.append(intR5)
        S = newS
        
    return library,intlib,intlib5

library,intlib,intlib5 = produce_series(100000,4,123,234,1000000)


#生成初始种群并计算适应度
size = 100   #种群规模
population = []    #种群

for i in range(size):
    chromosome = random.sample(intlib,6)
    result,adaption = solve(distribution=distribution,alpha=alpha,tao=tao,
                            chromo=chromosome,Net=Net,ODnum=ODnum,A=A,Lmax=Lmax,
                            pathgroup=pathgroup,pathcost=pathcost,matrix=matrix,c=c,b=b)                
    match = random.choice(library)
    population.append([chromosome,adaption,match,result])

    
flag = 0            ##########################

#开始进化
for i in range(100):                     #i为进化次数

#交叉
    population = sort(population,2)    #每个个体取一个01间的随机数并从小到大排序
    
    pop = copy.deepcopy(population[:20])   #复制前20个
    for i in [x for x in range(20) if x%2==0]:   #对前20个进行两两交叉
        point = random.choice(intlib5)    #随机选择的点位
        pop[i][0][point:],pop[i+1][0][point:] = pop[i+1][0][point:],pop[i][0][point:]
        population.append(pop[i])              #加入种群
        population.append(pop[i+1])

#突变
    popu = copy.deepcopy(population)#复制所有个体
    for i in range(size):
        for j in range(6):   #遍历每一个基因
            value = random.choice(library)
            if value<0.01: popu[i][0][j] = 1-popu[i][0][j]
        if population[i][0] != popu[i][0]:
            population.append(popu[i])        #加入种群
        
#生成新一代种群
    for i in range(size):
        population[i][3],population[i][1] = solve(distribution=distribution,alpha=alpha,tao=tao,
                                                  chromo=population[i][0],Net=Net,ODnum=ODnum,A=A,
                                                  Lmax=Lmax,pathgroup=pathgroup,pathcost=pathcost,
                                                  matrix=matrix,c=c,b=b)    #更新适应度和路径流量             
        population[i][2] = random.choice(library)                           #更新每个染色体对应的随机数
        if population[i][1] != 1000000000: flag = flag + 1                 #计算有可行解的次数

    population = sort(population,1)  #根据适应度从小到大排列
    population = population[:size]

#找最优解
index = 0
for i in range(size):
    if population[i][1]<population[index][1]: index = i
print(population[index])
flag

该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解


该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
该问题无可行解
[[0, 0, 1, 0, 1, 1], 5713750.0, 0.886886, [1000.0, 0, 4000.0, 0.0, 4000.0, 0, 4000.0, 0.0, 500.0, 0.0, 5500.0, 0.0, 2500.0, 1000.0, 500.0, 3500.0, 2500.0, 0, 4000.0, 3000.0, 2000.0, 0.0, 3000.0, 1000.0]]


10000

In [176]:
def solve(chromo):    #适应度函数
    return 7*chromo[0]+chromo[1]-8*chromo[0]*chromo[1]-8*chromo[2]+9*chromo[3]+7*chromo[4]-15*chromo[3]*chromo[4]+chromo[5]

a=[1,1,1,1,1,1]
def ssr(solve,a):
    b = solve(a)
    return b
ssr(solve,a)

-6

In [196]:
a = [1,2,3,4,5,6]
sum(a)

21